# Problem set 2: Descriptive economics

**Learning goals:**

- Using basic `pandas` DataFrame operations
- Clean and structure data
- Download real economic datasets using an API
- Merge and join data sets
- Compute summary statistics

**Table of contents**<a id='toc0_'></a>    
- 1. [Basic pandas](#toc1_)    
  - 1.1. [Dataframe](#toc1_1_)    
  - 1.2. [New variable](#toc1_2_)    
  - 1.3. [Indexing](#toc1_3_)    
  - 1.4. [Changing variables](#toc1_4_)    
  - 1.5. [Dropping observations and columns](#toc1_5_)    
  - 1.6. [Renaming](#toc1_6_)    
  - 1.7. [Income distribution](#toc1_7_)    
- 2. [National account identity](#toc2_)    
  - 2.1. [Download](#toc2_1_)    
  - 2.2. [Merge](#toc2_2_)    
  - 2.3. [Split-apply-combine-plot](#toc2_3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:

%pip install git+https://github.com/alemartinello/dstapi
%pip install fredapi
import numpy as np
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
plt.rcParams.update({'axes.grid':True,'grid.color':'black','grid.alpha':'0.25','grid.linestyle':'--'})
plt.rcParams.update({'font.size': 14})

from dstapi import DstApi

  Cloning https://github.com/alemartinello/dstapi to /private/var/folders/j0/qzh343x57q9djs607v5lcwkr0000gn/T/pip-req-build-s_a06214
  Running command git clone --filter=blob:none --quiet https://github.com/alemartinello/dstapi /private/var/folders/j0/qzh343x57q9djs607v5lcwkr0000gn/T/pip-req-build-s_a06214
  Resolved https://github.com/alemartinello/dstapi to commit d9eeb5a82cbc70b7d63b2ff44d92632fd77123a4
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'dstapi' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'dstapi'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for dstapi: filename=dstapi-0.2-py3-none-any.whl size=4128 sha256=8a11ed96cb6

## 1. <a id='toc1_'></a>[Basic pandas](#toc0_)

### 1.1. <a id='toc1_1_'></a>[Dataframe](#toc0_)

Modify the code below such that *income* and *consumption* are variables in the *df* DataFrame.

In [30]:
np.random.seed(1999)
 
N = 100
mydata = {}
mydata['id'] = range(N)
mydata['income'] = np.exp(np.random.normal(size=N))
mydata['consumption'] = np.sqrt(mydata['income'])
mydata['ratio'] = df['consumption'] / df['income']
df = pd.DataFrame(mydata)
df.head()

,id,income,consumption,ratio
0,0,0.727981,0.853218,1.172033
1,1,1.997831,1.413447,0.707490
2,2,0.276823,0.526140,1.900636
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513


In [3]:
display(df)

,id,income,consumption
0,0,0.727981,0.853218
1,1,1.997831,1.413447
2,2,0.276823,0.526140
3,3,1.481931,1.217346
4,4,1.235904,1.111712
...,...,...,...
95,95,0.201856,0.449284
96,96,2.368034,1.538842
97,97,2.389874,1.545922
98,98,1.488635,1.220096


### 1.2. <a id='toc1_2_'></a>[New variable](#toc0_)

Create a new variable *ratio* which is the ratio of consumption to income.

In [6]:
df['ratio'] = df['consumption'] / df['income']
df.head()

,id,income,consumption,ratio
0,0,0.727981,0.853218,1.172033
1,1,1.997831,1.413447,0.707490
2,2,0.276823,0.526140,1.900636
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513


### 1.3. <a id='toc1_3_'></a>[Indexing](#toc0_)

**Question a:** Select everybody: with an income above 1.

In [8]:
I = df['income'] > 1
df.loc[I, :].head()

,id,income,consumption,ratio
1,1,1.997831,1.413447,0.707490
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513
6,6,2.574032,1.604379,0.623294
7,7,2.475478,1.573365,0.635580


**Question b:** Select everybody with an income *above* 1 and a ratio *above* 0.7.

In [9]:
IR = (I) & (df['ratio'] > 0.7)
df.loc[I, :].head()

,id,income,consumption,ratio
1,1,1.997831,1.413447,0.707490
3,3,1.481931,1.217346,0.821459
4,4,1.235904,1.111712,0.899513
6,6,2.574032,1.604379,0.623294
7,7,2.475478,1.573365,0.635580


### 1.4. <a id='toc1_4_'></a>[Changing variables](#toc0_)

**Question a:** Set consumption equal to 0.5 if income is less than 0.5.

In [10]:
I=(df['income']<0.5)
df.loc[I, ['consumption']]=0.5
df['consumption'].mean()

np.float64(1.075479712048503)

**Question b:** Set consumption equal to income if income is less than 0.5.

In [13]:
I=(df['income']<0.5)
df.loc[I, ['consumption']]=df.loc[I, ['income']].values
df['consumption'].mean()


np.float64(1.0337728690050052)

### 1.5. <a id='toc1_5_'></a>[Dropping observations and columns](#toc0_)

Drop the *ratio* variable and all rows with an income above 1.5. After this, also drop the first 5 rows.

In [31]:
df_alt = df.copy()

print(f'before: {df.shape[0]} observations, {df.shape[1]} variables')
df = df.drop('ratio', axis=1)
I = df['income'] > 1.5
df = df.drop(df[I].index)
df = df.drop(df.loc[:5].index)
print(f'after: {df.shape[0]} observations, {df.shape[1]} variables')

before: 100 observations, 4 variables
after: 65 observations, 3 variables


### 1.6. <a id='toc1_6_'></a>[Renaming](#toc0_)

Rename *consumption* to *cons* and *income* to *inc*.

In [32]:
df = df.rename(columns={'income':'inc','consumption':'con'})
df.head()

,id,inc,con
8,8,0.582074,0.762938
10,10,0.932044,0.965424
12,12,0.356952,0.597454
13,13,0.379825,0.616299
16,16,0.700896,0.837195


In [33]:
N = 100
mydata = {}
mydata['id'] = range(N)
mydata['income'] = np.exp(np.random.normal(size=N))
mydata['consumption'] = np.sqrt(mydata['income'])

df_true = pd.DataFrame(mydata)

df_true = df_true.rename(columns={'income':'inc','consumption':'con'})
df_true.head()

,id,inc,con
0,0,0.599145,0.774044
1,1,0.469167,0.684957
2,2,1.435292,1.198037
3,3,2.504716,1.582629
4,4,0.156667,0.395811


### 1.7. <a id='toc1_7_'></a>[Income distribution](#toc0_)

Compute the share of income for each decile of the income distribution using the code below as a starting point.

In [50]:
deciles = df.quantile([0.1 * i for i in range(1, 10)])
df['decile'] = pd.cut(df_true['inc'], bins=[-np.inf] + list(deciles['inc']) + [np.inf], labels=range(1, 11))
income_share = df.groupby('decile')['inc'].sum() / df['inc'].sum()
display(income_share)

AssertionError: 

## 2. <a id='toc2_'></a>[National account identity](#toc0_)

### 2.1. <a id='toc2_1_'></a>[Download](#toc0_)

Consider the following dictionary definitions:

In [13]:
columns_dict = {}
columns_dict['TRANSAKT'] = 'variable'
columns_dict['PRISENHED'] = 'unit'
columns_dict['TID'] = 'year'
columns_dict['INDHOLD'] = 'value'

var_dict = {} # var is for variable
var_dict['P.1 Output'] = 'Y'
var_dict['P.3 Final consumption expenditure'] = 'C'
var_dict['P.3 Government consumption expenditure'] = 'G'
var_dict['P.5 Gross capital formation'] = 'I'
var_dict['P.6 Export of goods and services'] = 'X'
var_dict['P.7 Import of goods and services'] = 'M'

unit_dict = {}
unit_dict['2020-prices, chained values'] = 'real'
unit_dict['Current prices'] = 'nominal'

**Step 1:** Download all of table `nah1`.

In [54]:
nah1_api = DstApi('NAH1') 
params = nah1_api._define_base_params(language='en')
display(params)
nah1_true = nah1_api.get_data(params=params)

{'table': 'nah1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'TRANSAKT', 'values': ['*']},
  {'code': 'PRISENHED', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

**Step 2:** Rename the columns using `columns_dict` and replace data using `var_dict` and `unit_dict`.

In [15]:
# hint, nah1_true = nah1_true.rename(?)

# for key,value in var_dict.items():
#   nah1.variable.replace(?)

#for key,value in unit_dict.items():
#   nah1.unit.replace(?)

**Step 3:** Only keep rows where the variable is in `[Y, C, G, I, X, M]`. Afterwards convert the `value` column to a float.

In [16]:
# write you code here
# nah1.value = nah1.value.astype('float')

**Step 4:** Discuss what the following summary statistics show.

In [17]:
# nah1.groupby(['variable','unit']).describe()

**Step 5:** Sort the dataset by year

In [18]:
# nah1 = nah1.sort_values(by='?')
# nah1.head()

### 2.2. <a id='toc2_2_'></a>[Merge](#toc0_)

Load population data from Denmark Statistics:

In [77]:
BEFOLK1_api = DstApi('BEFOLK1')
params = BEFOLK1_api._define_base_params(language='en')
display(params)

{'table': 'befolk1',
 'format': 'BULK',
 'lang': 'en',
 'variables': [{'code': 'KØN', 'values': ['*']},
  {'code': 'ALDER', 'values': ['*']},
  {'code': 'CIVILSTAND', 'values': ['*']},
  {'code': 'Tid', 'values': ['*']}]}

In [91]:
for code in ['KØN','CIVILSTAND']:
    print(code)
    display(BEFOLK1_api.variable_levels(code,language='en'))
    print('')

KØN


,id,text
0,TOT,I alt
1,1,Mænd
2,2,Kvinder



CIVILSTAND


,id,text
0,TOT,I alt
1,U,Ugift
2,G,Gift/separeret
3,E,Enke/enkemand
4,F,Fraskilt


In [97]:
params['variables'][0]['values'] = ['TOT'] 
params['variables'][2]['values'] = ['TOT'] 
BEFOLK1 = BEFOLK1_api.get_data(params=params)
display(BEFOLK1.head())

,KØN,ALDER,CIVILSTAND,TID,INDHOLD
0,Total,"Age, total",Total,1999,5313577
1,Total,0 years,Total,1999,66205
2,Total,1 year,Total,1999,67983
3,Total,10 years,Total,1999,61241
4,Total,11 years,Total,1999,58580


In [99]:
BEFOLK1 = BEFOLK1.rename(columns={'TID':'year','INDHOLD':'population'})
BEFOLK1 = BEFOLK1.drop(columns=['KØN','CIVILSTAND'])
pop = BEFOLK1[BEFOLK1.ALDER == 'Age, total'].drop(columns=['ALDER'])
pop.head()

,year,population
0,1999,5313577
101,2018,5781190
202,1972,4975653
303,2008,5475791
404,1973,5007538


**Question a:** Merge the population and the national account data, so there is a new column called `population`. Use the **merge function**.

In [21]:
# hint, merged = pd.merge(?,?,how='?',on=[?])
# merged_true.tail(10)

**Question b:** Merge the population on again, so there is a new column called `population_alt`. Use the **join method**.

In [22]:
# pop_with_index = pop.set_index(?)
# pop_with_index = pop_with_index.rename(columns={'population':'population_alt'})
# merged_with_index = merged.set_index(?)
# merged_alt = merged_with_index.join(?)
# merged_alt.tail(10)

**Question c:** Plot GDP per capita and GDP per working-age (18-65) using the code below as a starting point.

In [ ]:
# ages = ?

# working_pop = BEFOLK1[BEFOLK1.ALDER.isin(?)].groupby('year').?
# working_pop = working_pop.drop(columns=['ALDER'])
# working_pop = working_pop.rename(columns={'population':'working_population'})

# merged = pd.merge(nah1, working_pop, how='left', on=['year'])
# merged = pd.merge(merged, pop, how='left', on=['year'])

### 2.3. <a id='toc2_3_'></a>[Split-apply-combine-plot](#toc0_)

Ensure the following code for a **split-apply-combine-plot** can run.

In [23]:
# # a. split
# nah1_true_grouped = nah1_true.groupby(['variable','unit'])
# nah1_true_grouped_first = nah1_true_grouped.value.first()
# nah1_true_grouped_first.name = 'first'

# # b. apply
# nah1_true = nah1_true.set_index(['variable','unit','year'])
# nah1_true = nah1_true.join(nah1_true_grouped_first,how='left',on=['variable','unit'])
# nah1_true = nah1_true.reset_index()

# # c. combine
# nah1_true['indexed'] = nah1_true['value']/nah1_true['first']

# # d. plot
# def plot(df,variable='indexed'):
#     df_indexed = df.set_index('year')
#     I = df_indexed.unit == 'real'
#     df_indexed[I].groupby(['variable'])[variable].plot(legend=True);
    
# plot(nah1_true)

**Question:** Implement the same split-apply-combine as above using `transform`.

In [24]:
def first(x): # select the first element in a series
    return x.iloc[0]

# nah1_alt = nah1.copy()
# grouped = nah1_alt.groupby(?)
#nah1_alt[?] = ?.transform(lambda x: ?)
#nah1_alt.head()

In [25]:
# plot(nah1_alt,variable='index_transform')